In [1]:
!git clone https://gitee.com/jianywu/ML_chapter7_dataset.git
!ls ML_chapter7_dataset  

Cloning into 'ML_chapter7_dataset'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.
final  lgb_baseline_version1_chapter7.ipynb  README.md


In [3]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

# Load data and data preprocessing

In [4]:
seed = 42 # for the same data division

kf = KFold(n_splits=5, random_state=seed,shuffle=True)
df_train = pd.read_csv('ML_chapter7_dataset/final/train_final.csv')
df_test = pd.read_csv('ML_chapter7_dataset/final/test_final.csv')

X_train = df_train.drop(columns=['loan_status']).values
Y_train = df_train['loan_status'].values.astype(int)
X_test = df_test.drop(columns=['loan_status']).values
Y_test = df_test['loan_status'].values.astype(int)

In [5]:
X_train.shape, Y_train.shape

((50000, 145), (50000,))

In [6]:
# split data for five fold

five_fold_data = []

for train_index, eval_index in kf.split(X_train):
    x_train, x_eval = X_train[train_index], X_train[eval_index]
    y_train, y_eval = Y_train[train_index], Y_train[eval_index]
    
    five_fold_data.append([(x_train, y_train), (x_eval, y_eval)])

# Algorithm

In [7]:
def get_model(param):
    model_list = []
    for idx, [(x_train, y_train), (x_eval, y_eval)] in enumerate(five_fold_data):
        print('{}-th model is training:'.format(idx))
        train_data = lgb.Dataset(x_train, label=y_train)
        validation_data = lgb.Dataset(x_eval, label=y_eval)
        bst = lgb.train(param, train_data, valid_sets=[validation_data])
        model_list.append(bst)
    return model_list

# train

In [8]:
param_base = {'num_leaves': 31, 'objective': 'binary', 'metric': 'binary', 'num_round':1000}

param_fine_tuning = {'num_thread': 8,'num_leaves': 128, 'metric': 'binary', 'objective': 'binary', 'num_round': 1000, 
                     'learning_rate': 3e-3, 'feature_fraction': 0.6, 'bagging_fraction': 0.8}

In [9]:
# base param train
param_base_model = get_model(param_base)

# param fine tuning
param_fine_tuning_model = get_model(param_fine_tuning)

0-th model is training:
[1]	valid_0's binary_logloss: 0.448173
[2]	valid_0's binary_logloss: 0.405734
[3]	valid_0's binary_logloss: 0.372961
[4]	valid_0's binary_logloss: 0.34683


/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Streaming output truncated to the last 5000 lines.
[5]	valid_0's binary_logloss: 0.499498
[6]	valid_0's binary_logloss: 0.497659
[7]	valid_0's binary_logloss: 0.49584
[8]	valid_0's binary_logloss: 0.494033
[9]	valid_0's binary_logloss: 0.492254
[10]	valid_0's binary_logloss: 0.491941
[11]	valid_0's binary_logloss: 0.490211
[12]	valid_0's binary_logloss: 0.488466
[13]	valid_0's binary_logloss: 0.486737
[14]	valid_0's binary_logloss: 0.485043
[15]	valid_0's binary_logloss: 0.484723
[16]	valid_0's binary_logloss: 0.483066
[17]	valid_0's binary_logloss: 0.481415
[18]	valid_0's binary_logloss: 0.479759
[19]	valid_0's binary_logloss: 0.478125
[20]	valid_0's binary_logloss: 0.477819
[21]	valid_0's binary_logloss: 0.476211
[22]	valid_0's binary_logloss: 0.474603
[23]	valid_0's binary_logloss: 0.473019
[24]	valid_0's binary_logloss: 0.471441
[25]	valid_0's binary_logloss: 0.4699
[26]	valid_0's binary_logloss: 0.468355
[27]	valid_0's binary_logloss: 0.466821
[28]	valid_0's binary_logloss: 0.4665

# test

In [10]:
def test_model(model_list):
    data = X_test
    five_fold_pred = np.zeros((5, len(X_test)))
    for i, bst in enumerate(model_list):
        ypred = bst.predict(data, num_iteration=bst.best_iteration)
        five_fold_pred[i] = ypred
    ypred_mean = (five_fold_pred.mean(axis=-2)>0.5).astype(int)
    return accuracy_score(ypred_mean, Y_test)

In [11]:
base_score = test_model(param_base_model)
fine_tuning_score = test_model(param_fine_tuning_model)

print('base: {}, fine tuning: {}'.format(base_score, fine_tuning_score))

base: 0.91584, fine tuning: 0.91762
